<a href="https://colab.research.google.com/github/Mojtaba-Choopani/huggingface-llm-course-fa-notebooks/blob/main/chapter3-FINE-TUNING-PERETRAINED_MODEL/section2-Fine-tuning-a-model-with-the-Trainer-API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a model with the Trainer API

<div dir="rtl">
  <p>
    کتابخانه <strong>Transformers</strong> کلاس <code>Trainer</code> را برای کمک به <strong>fine-tune</strong> مدل‌های پیش‌آموزش‌دیده فراهم می‌کند. پس از انجام تمام مراحل پیش‌پردازش داده‌ها، تنها چند مرحله باقی مانده برای تعریف <code>Trainer</code>.
  </p>
  <p>
    یکی از چالش‌ها سرعت کند اجرای <code>Trainer.train()</code> در <strong>CPU</strong> است که بهتر است از <strong>GPU</strong> استفاده شود.
  </p>
</div>


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

<div dir="rtl">
  <p>
    کد زیر در بخش قبلی اجرایی و توضیح داده شد:
  </p>
</div>


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

<div dir="rtl">
  <p>
    <strong>آموزش</strong>
  </p>
  <p>
    برای شروع آموزش، ابتدا باید کلاس <code>TrainingArguments</code> را تعریف کنید که هایپرپارامترهای مورد نیاز برای آموزش و ارزیابی را شامل می‌شود. تنها آرگومانی که باید مشخص کنید، دایرکتوری ذخیره مدل و چک‌پوینت‌ها است، و برای باقی موارد می‌توانید از مقادیر پیش‌فرض استفاده کنید.
  </p>
</div>


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

<div dir="rtl">
  <p>
    مرحله دوم، تعریف مدل است. مانند فصل قبلی، از کلاس <strong>AutoModelForSequenceClassification</strong> استفاده خواهیم کرد، با دو برچسب:
  </p>
</div>


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()